In [ ]:
import notebook_setup
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from gpdm import GPDM
from gpdm_pf import GPDM_PF
from filterbank import FilterBank

from dataset_utils.mocap_labels import RUN_TRIALS_TEST, WALK_TRIALS_TEST
import dataset_utils.select_joints as select_joints

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.6.1 (SDL 2.30.7, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
TEST_TRIALS = RUN_TRIALS_TEST + WALK_TRIALS_TEST
TRUTH_LABELS = [1] * len(RUN_TRIALS_TEST) + [0] * len(WALK_TRIALS_TEST)

reduced_columns = TEST_TRIALS[0].get_columns_for_joints(select_joints.WALKING_SIMPLIFIED_JOINTS)
DOFs = len(reduced_columns)
print(f"Number of DOFs: {DOFs}")


Number of DOFs: 35


In [3]:
d = 3 # latent space dimension
DOFs = len(reduced_columns)  # number of degrees of freedom

dyn_back_step = 1 # Number of time steps to look back in the dynamics GP

# Initial values for hyperparameters
y_lambdas_init = np.ones(DOFs)  # Signal standard deviation for observation GP
y_lengthscales_init = np.ones(d)  # Lengthscales for observation GP
y_sigma_n_init = 1e-2  # Noise standard deviation for observation GP

x_lambdas_init = np.ones(d)  # Signal standard deviation for latent dynamics GP
x_lengthscales_init = np.ones(dyn_back_step*d)  # Lengthscales for latent dynamics GP
x_sigma_n_init = 1e-2  # Noise standard deviation for latent dynamics GP
x_lin_coeff_init = np.ones(dyn_back_step*d + 1)  # Linear coefficients for latent dynamics GP

In [4]:
model_dir = Path().cwd() / 'models' / 'epochs_1k'
walk_config = model_dir / 'walk_gpdm_config.pth'
walk_state = model_dir / 'walk_gpdm_state.pth'

walk_gpdm =  GPDM(
        D=DOFs,
        d=d,
        dyn_target='full',
        dyn_back_step=dyn_back_step,
        y_lambdas_init=y_lambdas_init,
        y_lengthscales_init=y_lengthscales_init,
        y_sigma_n_init=y_sigma_n_init,
        x_lambdas_init=x_lambdas_init,
        x_lengthscales_init=x_lengthscales_init,
        x_sigma_n_init=x_sigma_n_init,
        x_lin_coeff_init=x_lin_coeff_init
    )
walk_gpdm.load(walk_config, walk_state, False)
#walk_gpdm_filter = GPDM_UKF(walk_gpdm)
#walk_gpdm_filter = GPDM_Filter(walk_gpdm)
walk_gpdm_filter = GPDM_PF(walk_gpdm, num_particles=100)




GPDM correctly loaded


In [5]:
run_config = model_dir / 'run_gpdm_config.pth'
run_state = model_dir / 'run_gpdm_state.pth'

run_gpdm =  GPDM(
        D=DOFs,
        d=d,
        dyn_target='full',
        dyn_back_step=dyn_back_step,
        y_lambdas_init=y_lambdas_init,
        y_lengthscales_init=y_lengthscales_init,
        y_sigma_n_init=y_sigma_n_init,
        x_lambdas_init=x_lambdas_init,
        x_lengthscales_init=x_lengthscales_init,
        x_sigma_n_init=x_sigma_n_init,
        x_lin_coeff_init=x_lin_coeff_init
    )

run_gpdm.load(run_config, run_state, False)
#run_gpdm_filter = GPDM_UKF(run_gpdm)
#run_gpdm_filter = GPDM_Filter(run_gpdm)
run_gpdm_filter = GPDM_PF(run_gpdm, num_particles=100)




GPDM correctly loaded


In [6]:
# Make a filter bank
filter_bank = FilterBank([walk_gpdm_filter, run_gpdm_filter], window_size=10)


In [7]:
# Iterate over the test trials and classify them

for i, trial in enumerate(TEST_TRIALS):
    label = TRUTH_LABELS[i]

    # Get the data in an array format
    data_arr = trial.as_dataframe()[reduced_columns].to_numpy(dtype=np.float32)[::5, :]

    filter_bank.reset()
    correct = 0
    total = 0

    # Run the filter bank on the trial
    for data in data_arr:
        filter_bank.update(data)
        predicted_label = filter_bank.get_most_likely()
        latest_lls = filter_bank.get_latest_log_likelihood()
        walk_ll = latest_lls[0]
        run_ll = latest_lls[1]

        if predicted_label == label:
            correct += 1
        
        total += 1

    accuracy = correct / total

    print(f"Trial {i}, Accuracy: {accuracy}")
    print(f"Trial {i} Final, True label: {label}, Predicted label: {predicted_label}, Walk LL: {walk_ll}, Run LL: {run_ll}")


Trial 0, Accuracy: 0.9666666666666667
Trial 0 Final, True label: 1, Predicted label: 1, Walk LL: -6.587240794543334, Run LL: 53.50625461487343
Trial 1, Accuracy: 1.0
Trial 1 Final, True label: 1, Predicted label: 1, Walk LL: -24.25125457137442, Run LL: 29.299724731379996
Trial 2, Accuracy: 1.0
Trial 2 Final, True label: 1, Predicted label: 1, Walk LL: -33.962650399433215, Run LL: 90.04984730096737
Trial 3, Accuracy: 1.0
Trial 3 Final, True label: 1, Predicted label: 1, Walk LL: -46.30429578648799, Run LL: 39.85175702859815
Trial 4, Accuracy: 1.0
Trial 4 Final, True label: 1, Predicted label: 1, Walk LL: -61.68530613906116, Run LL: 8.50847295142932
Trial 5, Accuracy: 1.0
Trial 5 Final, True label: 1, Predicted label: 1, Walk LL: -43.95723497890036, Run LL: 2.3045191406825376
Trial 6, Accuracy: 0.48484848484848486
Trial 6 Final, True label: 1, Predicted label: 0, Walk LL: 17.420386958785404, Run LL: 10.082493496068288
Trial 7, Accuracy: 1.0
Trial 7 Final, True label: 1, Predicted label: 